In [2]:
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from scipy import stats
import torch
from collections import defaultdict
from collections import Counter
from random import randint
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pickle

In [12]:
pos_data = pickle.load(open('pos_data.p', 'rb'))
neg_data = pickle.load(open('neg_data.p', 'rb'))

unigram_probs = pickle.load(open('unigram_probs.p', 'rb'))
vocab_size = len(unigram_probs)

central_words = []
contexts = []
neg_samples = []

for p in pos_data:
    central_words.append(p[0])
    contexts.append(p[1])
    
for n in neg_data:
    neg_samples.append(n[1])

In [13]:
print(central_words[0], contexts[0], neg_samples[0])

41 64 [44, 47, 19, 10, 79]


In [16]:
torch.set_default_tensor_type('torch.DoubleTensor')

class BayesianSG(nn.Module):

    def __init__(self,embedding_size, vocab_size):
        super(BayesianSG, self).__init__()
        self.w_embeddings = nn.Embedding(vocab_size, embedding_size)
        self.c_embeddings = nn.Embedding(vocab_size, embedding_size)
        
        self.dense_weights = nn.Linear(vocab_size*2, embedding_size, bias = False)
        
        self.muLinear = nn.Linear(embedding_size, embedding_size)
        self.sigmaLinear = nn.Linear(embedding_size, embedding_size)
        self.fLinear = nn.Linear(embedding_size, vocab_size)
        self.softm = nn.Softmax(dim=0)
        
        self.Location = nn.Linear(vocab_size,1, bias = False)
        self.Scale = nn.Linear(vocab_size,1, bias = False)
        
        
    def forward(self, central_words, contexts):
        
        out = torch.cat((central_words, contexts), 1) #concatenate dense vectors
        
        out = self.dense_weights(out)
        
        out = F.relu(out)
        
        out = torch.sum(out,0)
        
        mu_posterior = self.muLinear(out)
        
        sigma_posterior = self.sigmaLinear(out)
        
        sigma_posterior = F.softplus(sigma_posterior)
        
        dims = mu_posterior.shape
        
        epsilon = torch.distributions.multivariate_normal.MultivariateNormal \
        (torch.zeros(dims),torch.diag(torch.ones(dims)))
        
        z = mu_posterior + sigma_posterior * epsilon.sample()
        
        out = self.fLinear(z)
        
        probs = self.softm(out)
        
        mu_prior = self.Location(central_words)
        
        sigma_prior = self.Scale(central_words)
        sigma_prior = F.softplus(sigma_prior)
        
        return mu_posterior, sigma_posterior, mu_prior, sigma_prior
    
embedding_size = 10 
model = BayesianSG(embedding_size, vocab_size)

#mu_posterior, sigma_posterior, mu_prior, sigma_prior = model(central_words, contexts, neg_samples)